# 前処理大全

# 第一章 前処理とは

### データ分析の３つの前処理
- 表やグラフ作成用の前処理
- 教師なし学習へ入力するための前処理
- 教師あり学習へ入力するための前処理

### 前処理の流れ  
#### 1 データ構造を対象とした処理:大きなデータを扱うので、SQLが有利。  
- 対象データの抽出や結合、集約など  
 
#### 2 データ内容を対象とした処理：値の修正などのため、python有利。  
- データ内容の変更や、欠損値の補完など  
→ 集計やグラフの描写に使用  
- 機械学習モデルのために変換  
→ 教師なし学習用データ  
    
#### 3 データ構造を対象とした処理:機械学習用のライブラリの多いpythonを使用。  
- 学習、テストデータの分割  
→ 教師あり学習用データ  

## 第二章 抽出
データサイズを小さくするメリットは多いので、必要なデータのみを抽出するようにする。

### 注意事項
- iloc、ixなど値で指定する方法は不推奨、locや配列に文字列を使用する事を推奨。
- indexが無い場合、条件指定時に全探索する事になる。indexがあれば、必要範囲の指定となるので処理速度が向上。
- 条件指定抽出は、query関数が一番視認性が良いので推奨。
- データ量を指定の数に減らせるsample関数は重要であるが、指定する項目の選択によってはデータ割合に支障をきたすので注意が必要。

# 第三章 集約

集約方法は大きく分けると２つの方法がある  
- groupby    : 条件表現が豊富、但し記述量が多い。
- Window関数  : SQLが簡単に書けてオススメ！！ グループ毎に並び替えて順位づけする事に長けている。

**注意事項**  
- 集約処理とwindow関数の実行を同時にはできない。（SQLはできる)


### aggが便利
- agg({"指定先1":処理内容1,"指定先2":処理内容})と記述するだけで、複数の処理を行ってくれる。 
- １つの処理の時には、記述量が増えるので使わない。

# 第四章 結合

### マスターテーブルとレコードテーブル
- マスターテーブルは、顧客の情報などの共通情報を保管しており、顧客IDを持っている。
- レコードテーブルは、商品情報や店舗情報など様々な種類がある。

レコードテーブルにも顧客IDを所持するものもあり、マスターテーブルと結合することで、顧客情報の紐付けができる。

### 注意事項  
結合前になるべく不要なもの除くために抽出しておく。  

### 過去データの取扱　　
- 結合対象の期間を絞る。
- 結合した過去データに集約関数を利用する。 window関数が最適、但しpythonは実装されてないのでrolling関数を利用する。  
  rolling: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html

# 第五章 分割  
SQLでの作業は全く効率的でないためしない事。

## 時系列データにおける処理
- 単純な交差検証は有効ではない。未来のデータが混じっている可能性がある。
- 上記の対策として、期間をスライドさせたり、期間を追加していく検証方法が有効。  
- 期間を追加して行く場合は、追加する時期によりデータ量が変わってくるので、データ量の増加に伴う精度の関係も把握する必要がある。

**注意事項**
- pythonには時系列データを簡単に扱えるライブラリは無い。

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("awesomebook/data/month_mst.csv")
df["year_month"] = df["year_num"].astype(str) + "-" + df["month_num"].astype(str)
df["year_month"] = df["year_month"].sort_values(ascending=True)
df.sort_values(by="year_month")


train_window_start = 1
train_window_end   = 24
horizon = 12
skip    = 12

while True:
    test_window_end = train_window_end + horizon
    train = df[train_window_start:train_window_end]
    test  = df[(train_window_end + 1):test_window_end]
    if test_window_end >= len(df.index):
        break

    train_window_start += skip
    train_window_end += skip

# 第六章 生成

## 不均等データの調整

障害でないデータに対して、障害データが極端に少ないケースを不均等と言う。
不均等なデータは機械学習の予測精度が下がる要因となる可能性が多い。

## 不均等なデータへの対策
- 機械学習のモデル制作時に重みを与える。
- データを操作して不均等な状態を解除する。  
  -少ないデータを増やす : オーバーサンプリング  
  -多いデータを減らす   : アンダーサンプリング  
  -この両方を行う方法
  
## アンダーサンプリング
多い方のデータを減らすだけなので、簡単に実装出来ます。。  
但し、データを減らす事は勿体無いので、基本的にはオーバーサンプリングを行います。  
オーバーサンプリングで過学習を起こしそうな場合に、両方行うようにする事が望ましい。

## オーバーサンプリング  
データを増やす時に使う、増やす手法としてSMOTEが使いやすい。  
SMOTEは、障害データとその選択候補となるデータの中間に新たなデータを作る手法。

In [2]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(k_neighbors=5, random_state=1)

# 第七章 展開

## 横持ちへの変換
pythonでは、集約処理と横持ち変換が同時にできる**ピボットテーブル**という非常に強力な関数がある。  
indexにはキー要素、columnsには要素の種類をあらわすキー値、valuesはデータ要素の値となる対象の列を指定出来ます。  
またaggfunc引数に、valuesで指定された値をデータの要素の値に変換して指定出来ます。

## スパースマトリックスへの変換
ほとんどの値が0で、ごくわずかしか値が存在しない巨大な行列のことを指す。又の名を疎行列。  
縦持ちデータでは０を表現しないルールでコンパクトな配列だったものが、横持ちにする事で0が出現し膨大な列数となることがあります。  


# 第八章 数値型

In [3]:
reserve_master = pd.read_csv("./awesomebook/data/reserve.csv")
customer_master = pd.read_csv("./awesomebook/data/customer.csv")
production_master = pd.read_csv("./awesomebook/data/production_missing_category.csv")
reserve = reserve_master.copy()
customer = customer_master.copy()
production = production_master.copy()
reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100


## 対数化
入力値を対数に変換する処理

In [4]:
reserve["log"] = reserve["total_price"].apply(lambda x: np.log(x / 1000 + 1))
reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,4.587006
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,3.072693
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,3.543854
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,5.275049
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,4.235555


## カテゴリー化  

In [5]:
customer["age_rank"] = (np.floor(customer["age"] / 10) * 10).astype("category")
customer

,customer_id,age,sex,home_latitude,home_longitude,age_rank
0,c_1,41,man,35.092193,136.512347,40.0
1,c_2,38,man,35.325076,139.410551,30.0
2,c_3,49,woman,35.120543,136.511179,40.0
3,c_4,43,man,43.034868,141.240314,40.0
4,c_5,31,man,35.102661,136.523797,30.0
...,...,...,...,...,...,...
995,c_996,44,man,34.465648,135.373787,40.0
996,c_997,35,man,35.345372,139.413754,30.0
997,c_998,32,woman,43.062267,141.272126,30.0
998,c_999,48,woman,38.172800,140.464198,40.0


## 正規化  


値が１の場合と、値が1000の場合では同じ値でも重要度が変わってしまうので、同じ重要度で比較する手法。
- 平均を0にして、分散を1に変換する正規化。
- 最小値0と最大値1に収まるように値を置き換える正規化。

In [6]:
from sklearn.preprocessing import StandardScaler

reserve["people_num"] = reserve["people_num"].astype(float)

ss = StandardScaler()
result = ss.fit_transform(reserve[["people_num", "total_price"]])
reserve["people_num_normalized"] = [x[0] for x in result]
reserve["total_price_normalized"] = [x[1] for x in result]
reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,log,people_num_normalized,total_price_normalized
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4.0,97200,4.587006,1.300709,-0.053194
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2.0,20600,3.072693,-0.483753,-0.747822
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2.0,33600,3.543854,-0.483753,-0.629935
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4.0,194400,5.275049,1.300709,0.828240
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3.0,68100,4.235555,0.408478,-0.317080


## 外れ値の除去
極端に異なる値は、機械学習の精度を狂わせてしますので取り除く処理を行う。  
絶対値を標準偏差で割る事で、平均値から標準偏差の何倍離れているかを計算し、不等式を使って削除する。

In [230]:
reserve = reserve_master.copy()
reserve.shape

(4030, 10)

In [231]:
reserve["price_std"] = abs(reserve["total_price"] - np.mean(reserve["total_price"])) / np.std(reserve["total_price"])
reserve = reserve[reserve["price_std"] <= 3]
reserve

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_std
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.053194
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.747822
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,0.629935
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.828240
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.317080
...,...,...,...,...,...,...,...,...,...,...
4024,r4025,h_160,c_999,2017-03-11 11:56:05,2017-03-27,10:00:00,2017-03-30,1,37200,0.597289
4025,r4026,h_129,c_999,2017-06-27 23:00:02,2017-07-10,09:30:00,2017-07-11,2,16000,0.789536
4026,r4027,h_97,c_999,2017-09-29 05:24:57,2017-10-09,10:30:00,2017-10-10,2,41800,0.555575
4027,r4028,h_27,c_999,2018-03-14 05:01:45,2018-04-02,11:30:00,2018-04-04,2,74800,0.256323


## 主成分分析による次元圧縮

2つ又はそれ以上の次元を持つ値を元の値以下の次元に圧縮することを次元削減といい、  
次元削減によって求めた値は、元の値をどんなけ再現しているかを寄与率で表す。  
寄与率は90%が望ましいが、基本的に次元削減は機械学習の精度を低下させる要因となるが、  
見やすいグラフ化や、新しい次元から新しい発見が得られたりします。

In [232]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
pca_values = pca.fit_transform(production[["length","thickness"]])

print("累積寄与率: {0}".format(sum(pca.explained_variance_ratio_)))
print("各次元の寄与率: {0}".format(pca.explained_variance_ratio_))

pca_newvalues = pca.transform(production[["length", "thickness"]])

累積寄与率: 1.0
各次元の寄与率: [0.97897794 0.02102206]


## 数値の補完

### 欠損値の種類は下記の３種類がある。
- MCAR : 偶然に起きたランダムな欠損
- MAR  : 欠損した項目データに関係なく、他の項目データに依存した欠損
- NMAR : 欠損した項目に依存した欠損


### 6つの補完手法
- 定数での補完
- 計算値での補完(mean,median,mode)
- 予測値での補完
- 時系列の関係から補完
- 多重代入法
- 尤度法

MCARやＭＡＲには、多重代入法や尤度法が有効、NＭＡＲには有効な方法が確立されていない。  
また本来は再度、実験を行い結果を取り直す事が望ましい。

In [233]:
production_miss_m = pd.read_csv("./awesomebook/data/production_missing_num_4_redshift.csv")
production_miss_tb = production_miss_m.copy()

production_miss_num_m = pd.read_csv("./awesomebook/data/production_missing_num.csv")
production_miss_num = production_miss_num_m.copy()

print(production_miss_num["thickness"].value_counts(None))

None                  108
24.319347580497993      1
10.381541523229858      1
66.27033480134469       1
28.56729661968654       1
                     ... 
32.17162952758487       1
7.519130311473952       1
2.9703832545340965      1
9.144969710997136       1
10.995655308032106      1
Name: thickness, Length: 893, dtype: int64


In [234]:
# 定数補完
production_miss_num = production_miss_num_m.copy()

production_miss_num.replace("None", np.nan, inplace=True)
production_miss_num["thickness"].fillna(1, inplace=True)

print(production_miss_num["thickness"].value_counts(None))

1                     108
16.06299196760124       1
66.27033480134469       1
28.56729661968654       1
57.60912127508247       1
                     ... 
32.17162952758487       1
7.519130311473952       1
2.9703832545340965      1
9.144969710997136       1
10.995655308032106      1
Name: thickness, Length: 893, dtype: int64


In [235]:
# 計算値補完
production_miss_num = production_miss_num_m.copy()

production_miss_num.replace("None", np.nan, inplace=True)
production_miss_num["thickness"] = production_miss_num["thickness"].astype(float)
production_miss_num["thickness"].fillna(production_miss_num["thickness"].mean(), inplace=True)
print(production_miss_num["thickness"].value_counts(None))

19.470386    108
11.462704      1
10.136377      1
33.314305      1
29.855163      1
            ... 
27.472758      1
17.440967      1
17.150642      1
10.257870      1
4.784968       1
Name: thickness, Length: 893, dtype: int64


In [236]:
# pip install cvxpy

In [237]:
## !!fancyimputeがインストールできない
from fancyimpute import MICE

production_miss_num = production_miss_num_m.copy()

production_miss_num.replace("None", np.nan, inplace=True)

production_miss_num["thickness"] = production_miss_num["thickness"].astype(float64)
production_miss_num["type"]

ModuleNotFoundError: No module named 'fancyimpute'

# カテゴリ型　

men/womenや30代、などのデータ値で、２つのカテゴリしか撮らない値はフラグ値と呼び、bool型と呼びます。  
カテゴリ型は非線形な変化を表現できるが、機械学習において正確に学習するには大量のデータが必要になります。  
カテゴリ型となった数値は、カテゴリ間の関係性データは表現されません。

### ダミー変数化  
機械学習に対応していないカタログ値を対応しているフラグの集合値に変換する事をダミー変数化といい、生成したフラグをダミー変数といいます。  
ダミー変数化には、ダミーフラグを１つ減らす効果があるが、減らすことでデータの視認性が落ちつ場合もあるので注意が必要。



In [243]:
customer["sex"] = pd.Categorical(customer["sex"])
dummy_vars = pd.get_dummies(customer["sex"], drop_first=False)
dummy_vars

,man,woman
0,1,0
1,1,0
2,0,1
3,1,0
4,1,0
...,...,...
995,1,0
996,1,0
997,0,1
998,0,1


### カテゴリの集約    
複数の値を集約する事で、データの数を減らす事ができる。
- なるべく近い値同士を集約する（１０代＋６０代はNG、60代以上Good)
- 集約する際はマスターデータに行うようにすること。

In [244]:
customer["age_rank1"] = pd.cut(customer["age_rank"], bins=[0,10,20,30,40,50,60,np.inf], right=True)
customer

KeyError: 'age_rank'

In [251]:
# 書籍の解答
customer["age_rank2"] = pd.Categorical(np.floor(customer["age"]/10)*10)
customer["age_rank2"].cat.add_categories(["60以上"],inplace=True)
customer.loc[customer["age_rank2"].isin([60.0, 70.0, 80.0]), "age_rank2"] = "60以上"
customer["age_rank2"].cat.remove_unused_categories()
customer["age_rank2"].value_counts()

60以上    326
40.0    255
30.0    245
50.0    155
20.0     19
60.0      0
70.0      0
80.0      0
Name: age_rank2, dtype: int64

### カテゴリ値の組み合わせ
新たな値と組み合わせつ事でカテゴリ率を高める事ができる

In [252]:
customer = customer_master.copy()

customer["sex_age"] = pd.Categorical(customer[["sex", "age"]].apply(lambda x: f"{x[0]}_{np.floor(x[1] / 10) * 10}", axis=1))
customer.head()

,customer_id,age,sex,home_latitude,home_longitude,sex_age
0,c_1,41,man,35.092193,136.512347,man_40.0
1,c_2,38,man,35.325076,139.410551,man_30.0
2,c_3,49,woman,35.120543,136.511179,woman_40.0
3,c_4,43,man,43.034868,141.240314,man_40.0
4,c_5,31,man,35.102661,136.523797,man_30.0


### カテゴリ型の数値型
基本的には過学習の原因やデータの本来の意味を無くすこともあるので、利用はお勧めしない。  
方法としては、カテゴリ値毎の指標や極地/代表値/ばらつき具合を計算して利用する。



In [253]:
production = production_master.copy()
production["type"] = production["type"].fillna("Z")
production.isnull().sum()

type         0
length       0
thickness    0
fault_flg    0
dtype: int64

In [254]:
fault_cnt_per_type = production.query("fault_flg").groupby("type")["fault_flg"].count()

type_cnt = production.groupby("type")["fault_flg"].count()

production["type_fault_rete"] = production[["type", "fault_flg"]] \
    .apply(lambda x:(fault_cnt_per_type[x[0]] - int(x[1])) / (type_cnt[x[0]] - 1),axis=1)

production.head()

,type,length,thickness,fault_flg,type_fault_rete
0,E,274.027383,40.241131,False,0.066667
1,D,86.319269,16.906715,False,0.036458
2,E,123.940388,1.018462,False,0.066667
3,B,175.554886,16.414924,False,0.039216
4,B,244.934740,29.061081,False,0.039216


### カテゴリ型の補完 
数値型と同様の６種類の方法で実行できる。

In [255]:
production_miss_cat_m = pd.read_csv("./awesomebook/data/production_missing_category.csv")
production_miss_cat_tb = production_miss_cat_m.copy()
production_miss_cat_tb.head(5)

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [256]:
production_miss_cat_tb.replace("None", np.nan, inplace=True)

production_miss_cat_tb.head(5)

,type,length,thickness,fault_flg
0,E,274.027383,40.241131,False
1,D,86.319269,16.906715,False
2,E,123.940388,1.018462,False
3,B,175.554886,16.414924,False
4,B,244.934740,29.061081,False


In [257]:
from sklearn.neighbors import KNeighborsClassifier

# 欠損していない、しているデータの抽出
train = production_miss_cat_tb.dropna(subset=["type"], inplace=False)
test = production_miss_cat_tb.loc[production_miss_num_tb.index.difference(train.index),:]

kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train[["length", "thickness"]], train["type"])

test["type"] = kn.predict(test[["length", "thickness"]])
test["type"]

8      E
26     E
30     E
36     A
41     E
      ..
971    A
980    E
983    B
992    A
996    A
Name: type, Length: 100, dtype: object

# 第十章 日時型

In [293]:
reserve= reserve_master.copy()
reserve.dtypes

reserve_id           object
hotel_id             object
customer_id          object
reserve_datetime     object
checkin_date         object
checkin_time         object
checkout_date        object
people_num            int64
total_price           int64
price_std           float64
dtype: object

In [294]:
reserve["reserve_datetime"] = pd.to_datetime(reserve["reserve_datetime"], format="%Y-%m-%d %H:%M:%S")         # 日付時刻

reserve["reserve_datetime"].dt.date # 日付
# reserve["reserve_datetime"].dt.time # 時刻
# reserve["reserve_datetime"].dt.year # 年
# reserve["reserve_datetime"].dt.month # 月
# reserve["reserve_datetime"].dt.day   # 日
# reserve["reserve_datetime"].dt.hour  # 時間
# reserve["reserve_datetime"].dt.minute # 分
# reserve["reserve_datetime"].dt.second # 秒

0       2016-03-06
1       2016-07-16
2       2016-09-24
3       2017-03-08
4       2017-09-05
           ...    
4025    2017-06-27
4026    2017-09-29
4027    2018-03-14
4028    2016-04-16
4029    2016-06-06
Name: reserve_datetime, Length: 4030, dtype: object

## 日時の差分と増減

In [301]:
reserve["checkin_datetime"] = pd.to_datetime(reserve["checkin_date"] + reserve["checkin_time"] ,format="%Y-%m-%d%H:%M:%S")
reserve.head(1)

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_std,checkin_datetime
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.053194,2016-03-26 10:00:00


In [302]:
# 月単位での差分
(reserve["reserve_datetime"].dt.year * 12 + reserve["reserve_datetime"].dt.month) - (reserve["checkin_datetime"].dt.year *12 + reserve["checkin_datetime"].dt.month)

0       0
1       0
2      -1
3       0
4       0
       ..
4025   -1
4026   -1
4027   -1
4028   -1
4029   -1
Length: 4030, dtype: int64

In [304]:
# 日単位での差分
(reserve["reserve_datetime"] - reserve["checkin_datetime"]).astype("timedelta64[D]")

0      -20.0
1       -4.0
2      -25.0
3      -22.0
4      -17.0
        ... 
4025   -13.0
4026   -11.0
4027   -20.0
4028   -24.0
4029   -31.0
Length: 4030, dtype: float64

In [320]:
import datetime
reserve = reserve.drop(["checkin_datetime_2", "checkin_datetime_3"], axis=1)
reserve["checkin_date_2"] = reserve["checkin_datetime"].dt.date + datetime.timedelta(days=1)
reserve["checkin_date_3"] = reserve["checkin_datetime"].dt.date + datetime.timedelta(days=-1)
reserve.head()

,reserve_id,hotel_id,customer_id,reserve_datetime,checkin_date,checkin_time,checkout_date,people_num,total_price,price_std,checkin_datetime,checkin_date_2,checkin_date_3
0,r1,h_75,c_1,2016-03-06 13:09:42,2016-03-26,10:00:00,2016-03-29,4,97200,0.053194,2016-03-26 10:00:00,2016-03-27,2016-03-25
1,r2,h_219,c_1,2016-07-16 23:39:55,2016-07-20,11:30:00,2016-07-21,2,20600,0.747822,2016-07-20 11:30:00,2016-07-21,2016-07-19
2,r3,h_179,c_1,2016-09-24 10:03:17,2016-10-19,09:00:00,2016-10-22,2,33600,0.629935,2016-10-19 09:00:00,2016-10-20,2016-10-18
3,r4,h_214,c_1,2017-03-08 03:20:10,2017-03-29,11:00:00,2017-03-30,4,194400,0.828240,2017-03-29 11:00:00,2017-03-30,2017-03-28
4,r5,h_16,c_1,2017-09-05 19:50:37,2017-09-22,10:30:00,2017-09-23,3,68100,0.317080,2017-09-22 10:30:00,2017-09-23,2017-09-21
